In [ ]:
#| default_exp sparse.sparsifier

In [ ]:
#| include: false
from nbdev.showdoc import *
from fastai.vision.all import *

In [ ]:
#| export
import numpy as np
import torch
import torch.nn as nn
import pickle
from itertools import cycle
from fastcore.basics import store_attr, listify, true
from typing import Callable, Optional, Union, List, Tuple, Type
from fasterai.core.criteria import *

A sparse vector, as opposed to a dense one, is a vector which contains a lot of zeroes. When we speak about making a neural network sparse, we thus mean that the network's weight are mostly zeroes.

With fasterai, you can do that thanks to the `Sparsifier` class.

In [ ]:
#| export
class Sparsifier():
    "Class providing sparsifying capabilities"
    def __init__(self, 
                 model: nn.Module,                        # The model to sparsify
                 granularity: str,                        # Granularity of sparsification (e.g., 'weight', 'filter')
                 context: str,                            # Context for sparsification ('global' or 'local')
                 criteria: Criteria,                      # Criteria to determine which weights to keep
                 nm: bool = False,                        # Whether to use N:M sparsity pattern (forces 2:4 sparsity)
                 layer_type: Type[nn.Module] = nn.Conv2d  # Type of layers to apply sparsification to
    ):
        if nm == True: print('Sparsity automatically set to 50% with 2:4 pattern')
        store_attr()
        self._save_weights()
        self._reset_threshold()

    def sparsify_layer(self, 
                       m: nn.Module,                   # The layer to sparsify
                       sparsity: float,                # Target sparsity level (percentage)
                       round_to: Optional[int] = None  # Round to a multiple of this value
    ) -> None:
        "Apply sparsification to a single layer"
        scores    = self._compute_scores(m, sparsity)
        threshold = self._compute_threshold(scores, sparsity, round_to)
        mask      = self._compute_mask(scores, threshold)
        m.register_buffer('_mask', mask)
        self._apply(m)
        self.criteria.update_weights(m)

    def sparsify_model(self, 
                       sparsity: Union[float, List[float]],  # Target sparsity level(s)
                       round_to: Optional[int] = None        # Round to a multiple of this value
    ) -> None:
        "Apply sparsification to all matching layers in the model"
        self._reset_threshold()
        sparsity_list = listify(sparsity)
        if len(sparsity_list) > 1 and self.context != 'local': raise ValueError(f"A list of sparsities can only be used with 'local' context, not {self.context}")
        sparsities = cycle(sparsity_list) if len(sparsity_list)==1 else iter(sparsity_list)
        mods = list(self.model.modules())
        for k,m in enumerate(self.model.modules()):
            if isinstance(m, self.layer_type): 
                sp = next(sparsities)
                self.sparsify_layer(m, sp, round_to)
                if k+1 < len(mods) and isinstance(mods[k+1], nn.modules.batchnorm._BatchNorm): self.sparsify_batchnorm(m, mods[k+1])
                
    def sparsify_batchnorm(self, 
                          m: nn.Module,       # The layer before batch norm
                          bn: nn.Module       # The batch norm layer
    ) -> None:
        "Apply filter pruning to batch norm parameters if appropriate"
        mask = getattr(m, "_mask", None)
        if self.granularity == 'filter' and true(mask):
            bn.weight.data.mul_(mask.squeeze())
            bn.bias.data.mul_(mask.squeeze())
            
    def _apply_masks(self) -> None:
        "Apply all stored masks to model weights"
        for m in self.model.modules():
            if isinstance(m, self.layer_type):
                self._apply(m)
        
    def _apply(self, 
              m: nn.Module  # Module to apply mask to
    ) -> None:
        "Apply mask to a module's weights"
        mask = getattr(m, "_mask", None)
        if true(mask): m.weight.data.mul_(mask)
        if self.granularity == 'filter' and true(m.bias):
            if true(mask): m.bias.data.mul_(mask.squeeze())
    
    def _reset_weights(self, 
                      model: Optional[nn.Module] = None  # Model to reset (default: self.model)
    ) -> None:
        "Reset weights to their initial values"
        model = model or self.model
        for m in model.modules():
            if hasattr(m, 'weight'):
                init_weights = getattr(m, "_init_weights", m.weight)
                init_biases = getattr(m, "_init_biases", m.bias)
                with torch.no_grad():
                    if true(m.weight): m.weight.copy_(init_weights)
                    if true(m.bias): m.bias.copy_(init_biases)
                self._apply(m)
            if isinstance(m, nn.modules.batchnorm._BatchNorm): m.reset_parameters()
                
    def _save_weights(self) -> None:
        "Save initial weights of the model"
        for m in self.model.modules():
            if hasattr(m, 'weight'):              
                m.register_buffer("_init_weights", m.weight.clone())
                bias = getattr(m, 'bias', None)
                if true(bias): m.register_buffer("_init_biases", bias.clone())
                    
    def save_model(self, 
                  path: str,             # Path to save the model
                  model: Optional[nn.Module] = None  # Model to save (default: self.model)
    ) -> None:
        "Save model without sparsification buffers"
        model = model or self.model
        tmp_model = copy.deepcopy(model)
        self._reset_weights(tmp_model)
        self._clean_buffers(tmp_model)
        torch.save(tmp_model, path)

    def _clean_buffers(self, 
                      model: Optional[nn.Module] = None  # Model to clean (default: self.model)
    ) -> None:
        "Remove internal buffers used for sparsification"
        model = model or self.model
        for m in model.modules():
            if hasattr(m, 'weight'):
                if hasattr(m, '_mask'): del m._buffers["_mask"]
                if hasattr(m, '_init_weights'): del m._buffers["_init_weights"]
                if hasattr(m, '_init_biases'): del m._buffers["_init_biases"]
                    
    def _reset_threshold(self) -> None:
        "Reset the threshold used for global pruning"
        self.threshold=None
            
    def _rounded_sparsity(self, 
                         n_to_prune: int,  # Number of elements to prune
                         round_to: int     # Rounding value
    ) -> int:
        "Round the number of elements to keep to a multiple of round_to"
        return max(round_to*torch.ceil(n_to_prune/round_to), round_to)
    
    def _compute_scores(self, 
                       m: nn.Module,  # Module to compute scores for
                       sparsity: float # Target sparsity level
    ) -> torch.Tensor:
        "Compute importance scores for weights based on criteria"
        return self.criteria(m, self.granularity)
                
    def _compute_threshold(self, 
                          scores: torch.Tensor,  # Importance scores
                          sparsity: float,       # Target sparsity level
                          round_to: Optional[int] # Rounding value
    ) -> torch.Tensor:
        "Compute threshold for pruning, with optional rounding"
        if self.context == 'global':
            if self.threshold is None: 
                global_scores  = torch.cat([self.criteria(m, self.granularity).view(-1) for m in self.model.modules() if isinstance(m, self.layer_type)])
                self.threshold = torch.quantile(global_scores.view(-1), sparsity/100)   
        elif self.context == 'local': 
            self.threshold = torch.quantile(scores.view(-1), sparsity/100)
        else: raise ValueError(f'Invalid context: {self.context}. Must be "global" or "local"')
            
        if round_to:
            n_to_keep = sum(scores.ge(self.threshold)).squeeze()
            self.threshold = torch.topk(scores.squeeze(), int(self._rounded_sparsity(n_to_keep, round_to)))[0].min()
        return self.threshold
    
    def _compute_mask(self, 
                     scores: torch.Tensor,  # Importance scores
                     threshold: torch.Tensor # Threshold for pruning
    ) -> torch.Tensor:
        "Compute binary mask for weights based on scores and threshold"
        if self.nm == True: return self._apply_nm_sparsity(scores)
        if threshold > scores.max(): threshold = scores.max() # Make sure we don't remove every weight of a given layer
        return scores.ge(threshold).to(dtype=scores.dtype)

    def _apply_nm_sparsity(self, 
                          scores: torch.Tensor  # Importance scores
    ) -> torch.Tensor:
        "Apply 2:4 structured sparsity pattern (N:M sparsity where N=2, M=4)"
        out_channels, in_channels, kernel_height, kernel_width = scores.shape
    
        if in_channels % 4 != 0 or in_channels * kernel_height * kernel_width % 16 != 0:
            print(f"Skipping 2:4 sparsity, Cin * Kh * Kw is not a multiple of 16")
            return torch.ones_like(scores)
    
        blocked_scores = rearrange(scores, 'o (b c) h w -> h w o b c', c=4)
        threshold = blocked_scores.topk(k=2, dim=-1).values[..., -1:]
        mask = (blocked_scores >= threshold).float()
        return rearrange(mask, 'h w o b c -> o (b c) h w')

    def print_sparsity(self) -> None:
        total_params = 0
        total_zeros = 0
        
        print("\nSparsity Report:")
        print("-" * 80)
        print(f"{'Layer':<20} {'Type':<15} {'Params':<10} {'Zeros':<10} {'Sparsity':<10}")
        print("-" * 80)
        
        for k, m in enumerate(self.model.modules()):
            if isinstance(m, self.layer_type):
                zeros = torch.sum(m.weight == 0).item()
                total = m.weight.nelement()
                sparsity_pct = 100.0 * zeros / total if total > 0 else 0
                
                print(f"{f'Layer {k}':<20} {m.__class__.__name__:<15} "
                      f"{total:<10,d} {zeros:<10,d} {sparsity_pct:>8.2f}%")
                
                total_params += total
                total_zeros += zeros
        
        print("-" * 80)
        overall_sparsity = 100.0 * total_zeros / total_params if total_params > 0 else 0
        print(f"{'Overall':<20} {'all':<15} {total_params:<10,d} "
              f"{total_zeros:<10,d} {overall_sparsity:>8.2f}%")

In [ ]:
show_doc(Sparsifier)

---

[source](https://github.com/FasterAI-Labs/fasterai/tree/master/blob/master/fasterai/sparse/sparsifier.py#L17){target="_blank" style="float:right; font-size:smaller"}

### Sparsifier

>      Sparsifier (model:torch.nn.modules.module.Module, granularity:str,
>                  context:str, criteria:fasterai.core.criteria.Criteria,
>                  nm:bool=False,
>                  layer_type:Type[torch.nn.modules.module.Module]=<class
>                  'torch.nn.modules.conv.Conv2d'>)

*Class providing sparsifying capabilities*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| model | Module |  | The model to sparsify |
| granularity | str |  | Granularity of sparsification (e.g., 'weight', 'filter') |
| context | str |  | Context for sparsification ('global' or 'local') |
| criteria | Criteria |  | Criteria to determine which weights to keep |
| nm | bool | False | Whether to use N:M sparsity pattern (forces 2:4 sparsity) |
| layer_type | Type | Conv2d | Type of layers to apply sparsification to |

The `Sparsifier` class allows us to remove some weights, that are considered to be less useful than others. This can be done by first creating an instance of the class, specifying:

- The `granularity`, i.e. the part of filters that you want to remove. Typically, we usually remove weights, vectors, kernels or even complete filters.
- The `context`, i.e. if you want to consider each layer independently (`local`), or compare the parameters to remove across the whole network (`global`).
- The `criteria`, i.e. the way to assess the usefulness of a parameter. Common methods compare parameters using their magnitude, the lowest magnitude ones considered to be less useful.

User can pass a single layer to prune by using the  `Sparsifier.sparsify_layer` method.

In [ ]:
show_doc(Sparsifier.sparsify_layer)

---

[source](https://github.com/FasterAI-Labs/fasterai/tree/master/blob/master/fasterai/sparse/sparsifier.py#L32){target="_blank" style="float:right; font-size:smaller"}

### Sparsifier.sparsify_layer

>      Sparsifier.sparsify_layer (m:torch.nn.modules.module.Module,
>                                 sparsity:float, round_to:Optional[int]=None)

*Apply sparsification to a single layer*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| m | Module |  | The layer to sparsify |
| sparsity | float |  | Target sparsity level (percentage) |
| round_to | Optional | None | Round to a multiple of this value |
| **Returns** | **None** |  |  |

Most of the time, we may want to prune the whole model at once, using the `Sparsifier.prune_model` method, indicating the percentage of sparsity to you want to apply.

In [ ]:
show_doc(Sparsifier.sparsify_model)

---

[source](https://github.com/FasterAI-Labs/fasterai/tree/master/blob/master/fasterai/sparse/sparsifier.py#L45){target="_blank" style="float:right; font-size:smaller"}

### Sparsifier.sparsify_model

>      Sparsifier.sparsify_model (sparsity:Union[float,List[float]],
>                                 round_to:Optional[int]=None)

*Apply sparsification to all matching layers in the model*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sparsity | Union |  | Target sparsity level(s) |
| round_to | Optional | None | Round to a multiple of this value |
| **Returns** | **None** |  |  |

In some case, you may want to impose the remaining amount of parameters to be a multiple of a given number (e.g. 8), this can be done by passing the `round_to` parameter.

<br>

Also, instead of passing a single value of sparsity, a list of sparsities can also be provided. In that case, each value in the list is the sparsity that will be applied to all layers.

**Example**: I have a 4-layer network and want to remove half of the parameters from the layers 2 and 3, I can provide the list: `sparsity = [0, 50, 50, 0]`
